In [397]:
import pandas as pd
import numpy as np
import os


### Read the UN Comtrade data

The time range is that the trade data for most countries are updated till Sep 2023

In [516]:
# Specify the path to the folder containing CSV files
folder_path = '../data/raw/MonthlyTradeData'

# Get a list of all files in the folder
files = [file for file in os.listdir(folder_path)]

# Create an empty list to store DataFrames
dfs = []

# Loop through each CSV file and read it into a DataFrame
for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, encoding='unicode_escape')
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame if needed
monthly_TradeData = pd.concat(dfs)

# Filter the date 
monthly_TradeData = monthly_TradeData[(monthly_TradeData['Period']<=202309)]

# Keep the necessary columns
monthly_TradeData = monthly_TradeData[['Period','ReporterISO','ReporterDesc','PartnerISO','PartnerDesc','PrimaryValue']]


Pre-processing data

In [517]:

# Define a function to strip whitespace for categorical values
def strip_categorical(value):
    if isinstance(value, str):
        return value.strip()
    return value

def replace_and_delete_countries(df):
    # Apply the function to the entire DataFrame
    df = df.applymap(strip_categorical)
    
    # Replace "Other Asia, nes" (S19) to Taiwan,China (TWM)
    df['ReporterISO'] = df['ReporterISO'].replace('S19', 'TWN')
    df.loc[df['ReporterISO'] == 'TWN', 'ReporterDesc'] = 'China, Taiwan'
    
    df['PartnerISO'] = df['PartnerISO'].replace('S19', 'TWN')
    df.loc[df['PartnerISO'] == 'TWN', 'PartnerDesc'] = 'China, Taiwan'
    
    # Delete other desc containing ', nes'
    # Replace values containing ', nes' with NaN
    df.loc[df['ReporterDesc'].str.contains(', nes', case=False, na=False), 'ReporterDesc'] = np.nan
    df.loc[df['PartnerDesc'].str.contains(', nes', case=False, na=False), 'PartnerDesc'] = np.nan
    # Delete rows containing Nan
    df = df.dropna()
    
    # Delete 'Free Zone', 'Bunkers', 'Special Categories', 'World', 'Neutral zone' 
    to_delete = ['Free Zone','Free Zones','Bunkers','Special Categories','World','Neutral Zone']
    
    # Delete rows where source is in the list of countries
    df = df[~df['ReporterDesc'].isin(to_delete)]
    # Delete rows where target is in the list of countries
    df = df[~df['PartnerDesc'].isin(to_delete)]

    return df

# Apply the function on monthly_TradeData
monthly_TradeData = replace_and_delete_countries(monthly_TradeData)

In [518]:
monthly_TradeData.head()

,Period,ReporterISO,ReporterDesc,PartnerISO,PartnerDesc,PrimaryValue
0,200101,NOR,Norway,FIN,Finland,1087262.245
1,200101,CHE,Switzerland,FIN,Finland,942722.476
2,200101,FRA,France,THA,Thailand,7555251.056
3,200101,ITA,Italy,THA,Thailand,60642.913
4,200101,CHE,Switzerland,THA,Thailand,752574.318


In [519]:
monthly_TradeData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 545537 entries, 0 to 37901
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Period        545537 non-null  int64  
 1   ReporterISO   545537 non-null  object 
 2   ReporterDesc  545537 non-null  object 
 3   PartnerISO    545537 non-null  object 
 4   PartnerDesc   545537 non-null  object 
 5   PrimaryValue  545537 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 29.1+ MB


In [520]:
# Save to csv
monthly_TradeData.to_csv('../data/processed/monthly_TradeData.csv',index=False)

Get the country list for all countries appeared in monthly_TradeData

In [521]:
# Creaet Appendix for country name and ISO code
def get_country_list(df):
    df = df[['ReporterISO','ReporterDesc','PartnerISO','PartnerDesc']]

    # Obtain the countries in the links
    countries_code = pd.concat([df[['ReporterISO']].rename(columns={'ReporterISO': 'ISO'}),df[['PartnerISO']].rename(columns={'PartnerISO': 'ISO'})]).drop_duplicates()
    
    # Match company info with selected companies
    countries = pd.merge(countries_code,df,left_on='ISO', right_on='ReporterISO', how='left')
    countries = countries[['ISO','ReporterDesc']].rename(columns={'ReporterDesc':'Name'}).drop_duplicates()
    
    # Merge data
    countries = pd.merge(countries, df, left_on='ISO',right_on='PartnerISO', how='left')
    
    # Rename columns
    countries = countries[['ISO','PartnerDesc']].rename(columns={'PartnerDesc':'Name'}).drop_duplicates()
    
    return countries

# Apply function
countries = get_country_list(monthly_TradeData)

In [404]:
# Apply function
countries = get_country_list(monthly_TradeData)

In [522]:
countries.head()

,ISO,Name
0,DZA,Algeria
2068,AGO,Angola
3542,AZE,Azerbaijan
5407,ARG,Argentina
9169,AUS,Australia


In [523]:
countries.info()


<class 'pandas.core.frame.DataFrame'>
Index: 237 entries, 0 to 477318
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISO     237 non-null    object
 1   Name    237 non-null    object
dtypes: object(2)
memory usage: 5.6+ KB


In [524]:
# Save to csv file
countries.to_csv('../data/processed/countries.csv',index=False, encoding='utf')

### Get feature data

TPU -- Trade policy uncertainty index https://www.policyuncertainty.com/trade_cimpr.html

In [534]:
# Read data
tpu = pd.read_excel('../data/raw/tpu_web_latest.xlsx', sheet_name='TPU_MONTHLY', index_col=None)

# Select columns
tpu = tpu[['DATE','TPU']]

# Rename 'month' to 'period'
tpu = tpu.rename(columns={'DATE': 'period'})

# Keep 'period' matches the format '2000M01'
tpu['period'] = pd.to_datetime(tpu['period']).dt.strftime('%YM%m')

# Filter the 'period' when it is between 2000M01 and 2023M09
tpu = tpu[(tpu['period'] >= '2000M01') & (tpu['period'] <= '2023M09')]

In [535]:
tpu.info()

<class 'pandas.core.frame.DataFrame'>
Index: 285 entries, 480 to 764
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   period  285 non-null    object 
 1   TPU     285 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.7+ KB


GPR -- Geopolitical Risk Index https://www.matteoiacoviello.com/gpr.htm

In [537]:
# Read data
gpr = pd.read_excel('../data/raw/data_gpr_export.xls', sheet_name='Data', index_col=None)

# Rename 'month' to 'period'
gpr = gpr.rename(columns={'month': 'period'})

# Keep 'period' matches the format '2000M01'
gpr['period'] = pd.to_datetime(gpr['period']).dt.strftime('%YM%m')

# Filter the 'period' when it is between 2000M01 and 2023M09
gpr = gpr[(gpr['period'] >= '2000M01') & (gpr['period'] <= '2023M09')]

# don't keep country specific index
gpr = gpr[['period', 'GPR']]

In [538]:
gpr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 285 entries, 1200 to 1484
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   period  285 non-null    object 
 1   GPR     285 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.7+ KB


GEM -- Global economic monitor index https://databank.worldbank.org/source/global-economic-monitor-(gem)#

In [539]:
# Read ata
gem = pd.read_excel('../data/raw/P_Data_Extract_From_Global_Economic_Monitor_(GEM).xlsx', sheet_name='Data', 
index_col=None)
gem = gem.transpose().iloc[3:]

# Make the first row the columns
gem.columns = gem.iloc[0]
gem = gem[1:]

# Move the existing index to a regular column
gem.reset_index(inplace=True)

# Rename the column 'Series Code' to 'period'
gem = gem.rename(columns={'index': 'period'})


In [540]:
# Keep only the part before the blank in the rows under column 'period'
gem['period'] = gem['period'].str.split().str[0]

# Keep rows only when 'period' matches the format '2000M01' and is between 2000M01 and 2023M09
gem = gem[gem['period'].str.match(r'^\d{4}M\d{2}$')]
gem = gem[(gem['period'] >= '2000M01') & (gem['period'] <= '2023M09')]

# Reset the index
gem = gem.reset_index(drop=True)

# Replace '..' to NA
gem.replace('..', np.nan, inplace=True)

# Keep the columns
gem = gem[['period','CPTOTSAXN','DXGSRMRCHSACD', 'DMGSRMRCHSACD', 'IPTOTSAKD', 'IMPCOV', 'NEER', 'REER', 'RETSALESSA', 'TOTRESV' ,'UNEMPSA_']] 


In [541]:
gem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   period         285 non-null    object 
 1   CPTOTSAXN      285 non-null    float64
 2   DXGSRMRCHSACD  285 non-null    float64
 3   DMGSRMRCHSACD  285 non-null    float64
 4   IPTOTSAKD      285 non-null    float64
 5   IMPCOV         285 non-null    float64
 6   NEER           285 non-null    float64
 7   REER           285 non-null    float64
 8   RETSALESSA     285 non-null    float64
 9   TOTRESV        285 non-null    float64
 10  UNEMPSA_       285 non-null    float64
dtypes: float64(10), object(1)
memory usage: 24.6+ KB


Construct feature matrix

In [542]:
# Merge tpu, gpr, gem into the features data
merged_data = pd.merge(tpu, gpr, on='period')
features = pd.merge(merged_data, gem, on='period')

# save to csv
features.to_csv('../data/processed/features.csv', index=False)

In [543]:
features.head()

,period,TPU,GPR,CPTOTSAXN,DXGSRMRCHSACD,DMGSRMRCHSACD,IPTOTSAKD,IMPCOV,NEER,REER,RETSALESSA,TOTRESV,UNEMPSA_
0,2000M01,32.117348,64.457809,76.089042,484771.067659,784349.502592,9.967223e+11,2.476091,883.756875,104.048961,61.870157,1.942120e+06,8.614909
1,2000M02,28.175856,63.541721,76.318397,496700.992153,943444.352810,1.010232e+12,2.063868,896.988312,104.329262,62.339465,1.947145e+06,8.620393
2,2000M03,36.838506,50.101986,76.498010,497995.743462,846133.068963,1.014447e+12,2.330691,903.555596,104.515925,62.045241,1.972075e+06,8.500496
3,2000M04,35.281040,48.682739,76.454511,482444.395690,860083.949923,1.018283e+12,2.299383,910.134906,104.679618,60.628472,1.977663e+06,8.448069
4,2000M05,49.327563,79.482880,76.617165,506354.667550,894550.760808,1.029954e+12,2.225832,931.339312,105.262834,60.970381,1.991119e+06,8.425447


In [544]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   period         285 non-null    object 
 1   TPU            285 non-null    float64
 2   GPR            285 non-null    float64
 3   CPTOTSAXN      285 non-null    float64
 4   DXGSRMRCHSACD  285 non-null    float64
 5   DMGSRMRCHSACD  285 non-null    float64
 6   IPTOTSAKD      285 non-null    float64
 7   IMPCOV         285 non-null    float64
 8   NEER           285 non-null    float64
 9   REER           285 non-null    float64
 10  RETSALESSA     285 non-null    float64
 11  TOTRESV        285 non-null    float64
 12  UNEMPSA_       285 non-null    float64
dtypes: float64(12), object(1)
memory usage: 29.1+ KB


### Get the Iudustry Production data 

IP  https://fred.stlouisfed.org/series/IPHITEK2S

In [546]:
# Read data
ip = pd.read_csv('../data/raw/IPHITEK2S.csv', encoding='unicode_escape')

# Rename columns
ip = ip.rename(columns={'DATE': 'period','IPHITEK2S':'ip'})

# Keep rows only when 'period' matches the format '2000M01' and is between 2000M01 and 2023M09
ip['period'] = pd.to_datetime(ip['period']).dt.strftime('%YM%m')
ip = ip[(ip['period'] >= '2000M01') & (ip['period'] <= '2023M09')]


In [548]:
ip.info()

<class 'pandas.core.frame.DataFrame'>
Index: 285 entries, 396 to 680
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   period  285 non-null    object 
 1   ip      285 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.7+ KB


In [ ]:
# Save to csv
ip.to_csv('../data/processed/ip.csv', index = False)